**Cecilia** **Conde**


```
Scrip clase ternaria
```



# Librerías


In [1]:
%%bash
pip install duckdb
pip install jupysql
pip install duckdb-engine

Couldn't find program: 'bash'


Configuracion la extensión de %sql para el notebook, lo que nos va a permitir usar lenguaje SQL directamente sobre una celda

In [5]:
import duckdb as duckdb
import pandas as pd

# %load_ext sql
%reload_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb://

There's a new jupysql version available (0.10.14), you're running 0.10.12. To upgrade: pip install jupysql --upgrade
Deploy Shiny apps for free on Ploomber Cloud! Learn more: https://ploomber.io/s/signup


In [6]:
con = duckdb.connect()

 # Datos

In [7]:
#seteo carpeta de trabajo y archivo
dataset_path = 'C:/Users/mconde/dmeyf2024/Competencia_02/'
# dataset_path = '~/buckets/b1/datasets' ojo esto cuando tenga colab en la nube
dataset_file = 'competencia_02_crudo.csv.gz'

In [8]:
%%sql
create or replace table competencia_02_crudo as
select
    *
from read_csv_auto("{{dataset_path + dataset_file}}")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Success


Visualización de tabla de sql

In [ ]:
%%sql
select *
from competencia_02_crudo limit 10

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_madelantodolares,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo
0,249221109,201901,1,0,1,59,276,7597.55,47433.58,5654.59,...,0.0,1,0.0,-44919.57,3.23,7136,24336.99,13.0,0.0,1466.25
1,249221468,201901,1,0,1,51,13,738.16,-325.09,491.31,...,0.0,1,0.0,-30184.75,15.24,393,16345.79,8.0,0.0,1548.36
2,249223005,201901,1,0,1,46,178,1014.31,9434.15,417.36,...,0.0,1,0.0,-3730.14,0.00,2755,1181.65,3.0,0.0,3917.82
3,249228180,201901,1,0,1,64,296,1028.33,6873.80,1129.43,...,0.0,1,0.0,-13196.25,0.00,8194,1892.04,1.0,0.0,2017.56
4,249232117,201901,1,0,1,77,349,11617.61,43291.55,10726.08,...,0.0,1,0.0,-22073.65,0.00,7876,21902.85,6.0,0.0,891.48
5,249236712,201901,1,0,1,42,1,1746.17,1746.17,0.00,...,NaN,1,0.0,NaN,NaN,10,NaN,NaN,NaN,0.00
6,249237079,201901,1,0,1,62,117,19193.25,110961.55,3147.21,...,0.0,8,0.0,-3688.58,0.00,3531,257.94,1.0,0.0,586.50
7,249237446,201901,1,0,1,52,253,3268.57,20250.17,220.94,...,0.0,8,0.0,-5574.87,0.00,3222,-1.34,4.0,0.0,680.34
8,249239632,201901,1,0,1,50,239,990.51,8868.52,343.85,...,0.0,1,0.0,-18110.05,0.00,5054,1596.32,2.0,0.0,1114.35
9,249242331,201901,1,0,1,53,256,3429.46,27997.33,1153.68,...,0.0,1,0.0,-49544.20,116.27,5708,35766.21,18.0,0.0,2498.49


 # Función de Clase ternaria


In [ ]:
#este es el que hay que usar con los parametros del profe!!

In [13]:


%%sql
create or replace table competencia_02 as
with periodos as (
    select distinct foto_mes from competencia_02_crudo
), clientes as (
    select distinct numero_de_cliente from competencia_02_crudo
), todo as (
    select numero_de_cliente, foto_mes from clientes cross join periodos
), clase_ternaria as (
    select
        c.*
        , if(c.numero_de_cliente is null, 0, 1) as mes_0
        , lead(mes_0, 1) over (partition by t.numero_de_cliente order by foto_mes) as mes_1
        , lead(mes_0, 2) over (partition by t.numero_de_cliente order by foto_mes) as mes_2
        , if (mes_2 = 1, 'CONTINUA',
          if(mes_1 = 0 , 'BAJA+1',
          if(mes_2= 0,'BAJA+2', null))) as clase_ternaria
    from todo t
    left join competencia_02_crudo c using (numero_de_cliente, foto_mes)
)
select
  * EXCLUDE (mes_0, mes_1, mes_2)
from clase_ternaria
where mes_0 = 1;

-- Agregar el conteo de clase ternaria
select clase_ternaria, count(*) as cantidad_clientes
from competencia_02
group by clase_ternaria;


,clase_ternaria,cantidad_clientes
0,CONTINUA,4365464
1,None,329519
2,BAJA+2,19941
3,BAJA+1,20669


Revisamos que todo salga como esperamos

In [14]:
%sql select count(*) from competencia_02

,count_star()
0,4735593


Y vemos la cardinalidad de las clases por periodo

In [15]:
%%sql
PIVOT competencia_02
on clase_ternaria
USING count(numero_de_cliente)
GROUP BY foto_mes

,foto_mes,BAJA+1,BAJA+2,CONTINUA
0,201901,688,718,123346
1,201902,720,688,124391
2,201903,688,760,124988
3,201904,759,579,125658
4,201905,580,660,126419
5,201906,662,608,127916
6,201907,609,689,129426
7,201908,683,552,131429
8,201909,553,576,133185
9,201910,583,624,135475


In [19]:
%%sql
select * from competencia_02 limit 10

,numero_de_cliente,foto_mes,active_quarter,cliente_vip,internet,cliente_edad,cliente_antiguedad,mrentabilidad,mrentabilidad_annual,mcomisiones,...,Visa_fultimo_cierre,Visa_mpagado,Visa_mpagospesos,Visa_mpagosdolares,Visa_fechaalta,Visa_mconsumototal,Visa_cconsumos,Visa_cadelantosefectivo,Visa_mpagominimo,clase_ternaria
0,249667857,201901,1,0,1,77,44,3549.77,26235.50,1336.82,...,22,20187.82,-19947.41,0.0,1177,13082.33,11,0,1208.19,CONTINUA
1,249667857,201902,1,0,1,77,45,3064.96,27805.96,1204.60,...,22,23509.10,-20187.82,0.0,1205,16030.44,12,0,1372.41,CONTINUA
2,249667857,201903,1,0,1,77,46,3933.42,29754.83,1325.10,...,25,18783.06,-23509.10,0.0,1236,14031.44,12,0,1102.62,CONTINUA
3,249667857,201904,1,0,1,77,47,5016.42,32881.14,1378.13,...,20,19062.68,-18783.06,0.0,1266,13431.42,11,0,1231.65,CONTINUA
4,249667857,201905,1,0,1,77,48,0.00,0.00,0.00,...,23,20934.23,-19062.68,0.0,1297,14174.00,10,0,1325.49,CONTINUA
5,249667857,201906,1,0,1,77,49,5398.26,40238.53,1378.50,...,25,23668.03,-20934.23,0.0,1327,16980.37,14,0,1501.44,CONTINUA
6,249667857,201907,1,0,1,78,50,5128.96,42692.96,1485.41,...,21,25771.40,-23668.03,0.0,1358,20258.23,15,0,1595.28,CONTINUA
7,249667857,201908,1,0,1,78,51,4973.30,45599.92,1832.68,...,24,25898.91,-25771.40,0.0,1389,18306.01,15,0,1994.10,CONTINUA
8,249667857,201909,1,0,1,78,52,4822.70,48395.34,2137.21,...,26,29407.15,-25898.91,0.0,1419,21658.91,13,0,1970.64,CONTINUA
9,249667857,201910,1,0,1,78,53,0.00,0.00,0.00,...,22,34071.50,-29407.15,0.0,1450,26525.64,14,0,1782.96,CONTINUA


Y finalmente almacenamos el nuevo dataset para ser usado a continuación

In [20]:
print(dataset_path)

C:/Users/mconde/dmeyf2024/Competencia_02/


In [21]:
%%sql
COPY competencia_02 TO 'C:/Users/mconde/dmeyf2024/Competencia_02/competencia_02.csv.gz' (FORMAT CSV, HEADER)

,Success
